# Principal Component Regression (PCR)

This notebook implements **Principal Component Regression (PCR)** from scratch and helps to visualise the difference from the **Ordinary Least Square(OLS) Linear Regression**.

# Defining Principal Component Regression (PCR)

Going by the definition, **Principal Component Regression (PCR)** is an algorithm for **reducing the multi-collinearity of datatsets.**

# Problems with using multi-variate Linear Regression on these datasets

### High Variance Issue

In multi-variate Linear Regression, it may seem that the accuracy is very high on the training set and hence we have trained a brilliant model. But in reality, our model have overfit the training set and there is a high variance present.

Such a model is likely to perform extremely worse on predictions with new examples and have a very low test accuracy.

# Benefits of using PCR on these datasets

Applying **PCR** on such multi-collinear datasets has the following benefits:

**PCR** adds a slight bias to the dataset and reduces the degree of overfitting.

**PCR** is basically applying **PCA** to the dataset to obtain the **Principal Components (PCs)**. Then **Ordinary Least Square Linear Regression** is done on these obtain **Principal Components (PCs)**.

# Functioning of PCR

In **PCR**, instead of regressing the dependent variable on the independant variables directly, the principal components of the independant variables are used.

We apply **PCA** to the dataset to obtain the **Principal Components (PCs)**. **PCA** is a **Dimensionality Reduction Technique**. It combines two or more features to develop new features called **Principal Components (PCs)**. It combines features such that they can suite the target variable in a suitable way.

# PCA: Is it a Feature Selection Technique ?

People often confuse **PCA** as a **Feature Selection Technique**.

**PCA is not a Feature Selection Technique**. PCA doesn't select a set of fewer features from all those available in order to reduce the number of features. 

**PCA actually combines all the features to develop Principal Components (PCs), which are entirely different from the actual features.** 

These **Principal Components (PCs)** suite the target variable with high accuracy.

# Let's now implement PCR and OLS Linear Regression

We will implement both **PCR** and **Ordinary Least Square(OLS) Limear Regression** on the given dataset and will perform a quantitative comparision.

## Import libraries

First of all we need to import all the necessary libraries. We will be using the following libraries:

**numpy**: python library used for scientific computing

In [94]:
# import all necessary libraries

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

## Load the dataset

In [95]:
hitters = pd.read_csv("Hitters.csv", sep = ",")
print("hitters.shape:  {}".format(hitters.shape))

hitters.shape:  (322, 20)


## Familiarisation with the dataset

The dataset contains information about 322 batsmen( *Hitters* ).

The dependant variable is the **Salary**.

In [96]:
# let's have a look at the first 10 rows of our dataset
hitters.head(n = 10)

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N
5,594,169,4,74,51,35,11,4408,1133,19,501,336,194,A,W,282,421,25,750.0,A
6,185,37,1,23,8,21,2,214,42,1,30,9,24,N,E,76,127,7,70.0,A
7,298,73,0,24,24,7,3,509,108,0,41,37,12,A,W,121,283,9,100.0,A
8,323,81,6,26,32,8,2,341,86,6,32,34,8,N,W,143,290,19,75.0,N
9,401,92,17,49,66,65,13,5206,1332,253,784,890,866,A,E,0,0,0,1100.0,A


## Preprocessing

Before using any dataset, we should perform the necessary preprocessing steps.

In the *Hitters.csv* dataset, values for **Salary** is missing for few examples. We need to remove them from the dataset.

We also need to create dummy variables for *qualitative features* like **League**, **Division** and **NewLeague**.

In [97]:
# Drop rows missing data -> having at least one NaN value
hitters = hitters.dropna(axis = 0, how = 'any', inplace = False)

In [98]:
# shape of the modified dataset
print("hitter.shape: {}".format(hitters.shape))

hitter.shape: (263, 20)


In [99]:
# 59 rows having at least one NaN value have been dropped

# let's look at the first 10 rows of our modified dataset
hitters.head(n = 5)

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N
5,594,169,4,74,51,35,11,4408,1133,19,501,336,194,A,W,282,421,25,750.0,A


In [100]:
# create dummy variables for qualitative features like League, Division and NewLeague
qualitative_features = ["League", "Division", "NewLeague"]

hitters = pd.get_dummies(hitters, columns = qualitative_features)

In [101]:
# let's look at the first 5 rows of our dataset
hitters.head(n = 5)

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,...,PutOuts,Assists,Errors,Salary,League_A,League_N,Division_E,Division_W,NewLeague_A,NewLeague_N
1,315,81,7,24,38,39,14,3449,835,69,...,632,43,10,475.0,0,1,0,1,0,1
2,479,130,18,66,72,76,3,1624,457,63,...,880,82,14,480.0,1,0,0,1,1,0
3,496,141,20,65,78,37,11,5628,1575,225,...,200,11,3,500.0,0,1,1,0,0,1
4,321,87,10,39,42,30,2,396,101,12,...,805,40,4,91.5,0,1,1,0,0,1
5,594,169,4,74,51,35,11,4408,1133,19,...,282,421,25,750.0,1,0,0,1,1,0


**Note**: Since **League** has two values **A** and **N**, two rows are created and assigned values 0 or 1 based on which value was originally present for the variable **League**. Similar is the case for the other two columns.

## Create the Train and Test Datasets

After preprocessing has been completed, we need to divide the dataset into two subsets: **Train** and **Test** datasets.

The **Train Dataset** is used to train our model.

The **Test Dataset** is used to evaluate our model.

### Shuffle the rows

In [102]:
# shuffle
hitters = hitters.sample(frac = 1)

### Split the dataset in X(features) and Y(labels)

In [103]:
# features
X = hitters.drop('Salary', axis = 1)

# labels
Y = hitters.Salary

### Split into X_train, X_test and Y_train, Y_test

In [104]:
# Train -> 80 % and Test -> 20 % of the total dataset
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [105]:
# verification
print("X_train.shape: {}".format(X_train.shape))
print("X_test.shape: {}".format(X_test.shape))

print("Y_train.shape: {}".format(Y_train.shape))
print("Y_test.shape: {}".format(Y_test.shape))

X_train.shape: (210, 22)
X_test.shape: (53, 22)
Y_train.shape: (210,)
Y_test.shape: (53,)


Train and Test Datasets are ready. 

Verified: Train Set -> 80 % and Test Set -> 20 % of the complete dataset

## Linear Regression

Let's now apply **Ordinary Least Square(OLS) Linear Regression** to the dataset.

### Define the model

In [106]:
# define the multi-variable Linear Regression Model
linear_regression = LinearRegression()

### Fit the model

In [107]:
# fit the model to X_train and Y_train ( training the model)
model = linear_regression.fit(X_train, Y_train)

### Predict 

In [108]:
# prediction with X_test
Y_pred = model.predict(X_test)

### Accuracy of our model

In [109]:
print("Training Set Accuracy Score:  {}".format(model.score(X_train, Y_train)))
print("Test Set Accuracy Score:  {}".format(model.score(X_test, Y_test)))

Training Set Accuracy Score:  0.5799927351178351
Test Set Accuracy Score:  0.18308569360911153


### Report

We used multi-variate **OLS Linear Regression** and got the **Test Set Accuracy Score:  0.038262328683979274**.

Let's use **Principal Component Regression (PCR)** and see if we could increase our accuracy.

## Principal Component Regression (PCR)

Let's now apply PCR to the dataset.

**PCR** is implemented in two steps:

1. Perform **PCA** on the dataset and get the **Principal Components(PCs)**.
2. Perform **OLS Linear Regression** on these **Principal Components(PCs)**.

## PCA to get the PCs

### Principal Components(PCs) Count

We could run **PCA** and get the **PCs** to perform **Linear Regression**.

But the question arises, **How many Principal Components do we need to consider?**

It's obvious that this can't be guessed beforehand and so we will deal with this problem in a later secction of this notebook.

For this section, we will take **10 PCs** to implement **PCR**.

### PCA to get 10 PCs

### Center and Scale the data

Before performing PCA, it's important to preprocess the data for better functioning.

**Centering**:  After centering, the average value of each feature will be 0.

**Scaling**:  After scaling, the standard deviation of each feature will be 1.

In [110]:
# preprocessing X_train and X_test
scalar = StandardScaler()
scalar.fit(X_train)

X_train_preprocess = scalar.transform(X_train)
X_test_preprocess = scalar.transform(X_test)

### Create a PCA object 

In [111]:
# create a PCA object with PCs count = 10
pca = PCA(n_components = 10)

### Fit the dataset

In [112]:
# fit X_train_preprocess to X_train_pca and X_test_preprocess to X_test_pca

pca.fit(X_train_preprocess)
X_train_pca = pca.transform(X_train_preprocess)

pca.fit(X_test_preprocess)
X_test_pca = pca.transform(X_test_preprocess)

### Familiarisation with the obtained PCs

In [113]:
# let's compare the shapes of X_train and X_train_pca
print("X_train.shape: {}".format(X_train.shape))
print("X_train_pca.shape: {}".format(X_train_pca.shape))

X_train.shape: (210, 22)
X_train_pca.shape: (210, 10)


In [114]:
# let's compare the shapes of X_test and X_test_pca
print("X_test.shape: {}".format(X_test.shape))
print("X_test_pca.shape: {}".format(X_test_pca.shape))

X_test.shape: (53, 22)
X_test_pca.shape: (53, 10)


We can see that the previously 210 features have been replaced with the obtained 10 PCs.

Let's now have a look on the variance contributed by each of the PCs.

In [115]:
for pc, variance_ratio in enumerate(pca.explained_variance_ratio_):
    print("Variance Ratio for Principal Component {}:  {}".format(pc+1, variance_ratio))

Variance Ratio for Principal Component 1:  0.3691492977522917
Variance Ratio for Principal Component 2:  0.2062222106523079
Variance Ratio for Principal Component 3:  0.15759793981168593
Variance Ratio for Principal Component 4:  0.08271732569284601
Variance Ratio for Principal Component 5:  0.05366456883530678
Variance Ratio for Principal Component 6:  0.03881300860109299
Variance Ratio for Principal Component 7:  0.027189968267227724
Variance Ratio for Principal Component 8:  0.020408129821324023
Variance Ratio for Principal Component 9:  0.01853554408769727
Variance Ratio for Principal Component 10:  0.008153735713847223


## Performing Linear Regression on the obtained PCs

### Fit the model

In [116]:
# fit a Linear Regression model to PCs and Y_train
PCA_model = linear_regression.fit(X_train_pca, Y_train)

### Predict

In [117]:
# predict using X_test_pca
Y_pred_pca = PCA_model.predict(X_test_pca)

### Accuracy of our model

In [118]:
print("Training Set Accuracy Score:  {}".format(PCA_model.score(X_train_pca, Y_train)))
print("Test Set Accuracy Score:  {}".format(PCA_model.score(X_test_pca, Y_test)))

Training Set Accuracy Score:  0.5040301908139364
Test Set Accuracy Score:  0.0844824433850585


**Note**: We can see that the accuracy for our model has dropped for both the Train and Test Set. 

This indicates that 10 Principal Components aren't enough to account for all the features